<a href="https://colab.research.google.com/github/ryunino/competition-code/blob/master/signate_JLeagueAttendancePredictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://signate.jp/competitions/27

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import random
from sklearn.metrics import roc_auc_score
from IPython.display import Image
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import math
from scipy import stats

%cd /content/drive/MyDrive/

Mounted at /content/drive
/content/drive/MyDrive


In [ ]:
#print("train")
df_train_base = pd.read_csv("./dsws/motodata/j-league/train.csv", sep=",", header=0, quotechar="\"")
df_train_add = pd.read_csv("./dsws/motodata/j-league/train_add.csv", sep=",", header=0, quotechar="\"")
#display(df_train_base.head(2))
#display(df_train_add.head(2))

#print("test")
df_test = pd.read_csv("./dsws/motodata/j-league/test.csv", sep=",", header=0, quotechar="\"")
#display(df_test.head(2))

#print("condition")
df_condition_base = pd.read_csv("./dsws/motodata/j-league/condition.csv", sep=",", header=0, quotechar="\"")
df_codition_add =  pd.read_csv("./dsws/motodata/j-league/condition_add.csv", sep=",", header=0, quotechar="\"")
#display(df_condition_base.head(2))
#display(df_codition_add.head(2))

#print("stadium")
df_stadium = pd.read_csv("./dsws/motodata/j-league/stadium.csv", sep=",", header=0, quotechar="\"")

#追加データの結合
df_train = pd.concat([df_train_base, df_train_add])
df_codition = pd.concat([df_condition_base, df_codition_add])

# train, testとconditionを結合する
df_train_all = pd.merge(df_train, df_codition, how="inner", on="id")
df_test_all = pd.merge(df_test, df_codition, how="inner", on="id")

# 結合したものに、stadiumを結合して上書きする
df_train_all = pd.merge(df_train_all, df_stadium, how="inner", left_on="stadium", right_on="name")
df_test_all = pd.merge(df_test_all, df_stadium, how="inner", left_on="stadium", right_on="name")

#print("train")
display(df_train_all.head(2))
#print("test")
display(df_test_all.head(2))

for i in df_train_all.columns:
  print(f'{i}' ,df_train_all[f'{i}'].unique())

,id,y,year,stage,match,gameday,time,home,away,stadium,...,away_05,away_06,away_07,away_08,away_09,away_10,away_11,name,address,capa
0,13994,18250,2012,Ｊ１,第１節第１日,03/10(土),14:04,ベガルタ仙台,鹿島アントラーズ,ユアテックスタジアム仙台,...,アレックス,青木 剛,増田 誓志,小笠原 満男,本山 雅志,大迫 勇也,ジュニーニョ,ユアテックスタジアム仙台,宮城県仙台市泉区七北田字柳78,19694
1,14012,11831,2012,Ｊ１,第３節第１日,03/24(土),13:03,ベガルタ仙台,大宮アルディージャ,ユアテックスタジアム仙台,...,下平 匠,青木 拓矢,上田 康太,渡邉 大剛,曺 永哲,東 慶悟,ラファエル,ユアテックスタジアム仙台,宮城県仙台市泉区七北田字柳78,19694


,id,year,stage,match,gameday,time,home,away,stadium,tv,...,away_05,away_06,away_07,away_08,away_09,away_10,away_11,name,address,capa
0,15822,2014,Ｊ１,第１８節第１日,08/02(土),19:04,ベガルタ仙台,大宮アルディージャ,ユアテックスタジアム仙台,スカパー！／スカパー！プレミアムサービス,...,高橋 祥平,橋本 晃司,和田 拓也,家長 昭博,富山 貴光,ムルジャ,ズラタン,ユアテックスタジアム仙台,宮城県仙台市泉区七北田字柳78,19694
1,15840,2014,Ｊ１,第２０節第１日,08/16(土),19:04,ベガルタ仙台,清水エスパルス,ユアテックスタジアム仙台,スカパー！／スカパー！プレミアムサービス,...,イ キジェ,本田 拓也,六平 光成,大前 元紀,河井 陽介,高木 俊幸,ノヴァコヴィッチ,ユアテックスタジアム仙台,宮城県仙台市泉区七北田字柳78,19694


id [13994 14012 14031 ... 16075 16178 15306]
y [18250 11831 13147 ...  3807  1921  3940]
year [2012 2013 2014]
stage ['Ｊ１' 'Ｊ２']
match ['第１節第１日' '第３節第１日' '第５節第１日' '第７節第１日' '第１０節第１日' '第１４節第１日' '第１６節第１日'
 '第１９節第１日' '第２０節第１日' '第２２節第１日' '第２４節第１日' '第２６節第１日' '第２８節第１日' '第２９節第１日'
 '第３１節第１日' '第３３節第１日' '第１１節第１日' '第１７節第１日' '第２３節第１日' '第２５節第２日' '第２７節第１日'
 '第５節第２日' '第８節第１日' '第１２節第１日' '第１４節第２日' '第６節第１日' '第８節第３日' '第１３節第１日'
 '第１８節第１日' '第３０節第１日' '第３０節第２日' '第３２節第１日' '第７節第２日' '第１５節第１日' '第９節第２日'
 '第２１節第１日' '第２５節第１日' '第３５節第１日' '第３７節第１日' '第３９節第１日' '第４１節第１日' '第４節第１日'
 '第３節第２日' '第９節第１日' '第６節第２日' '第２節第１日' '第１２節第２日' '第４０節第１日' '第４２節第１日'
 '第１３節第２日' '第１５節第２日' '第２８節第２日' '第３４節第１日' '第１節第２日' '第８節第２日' '第２６節第２日'
 '第１７節第２日' '第３７節第２日' '第３８節第１日' '第３５節第２日' '第１９節第２日' '第３６節第１日' '第１８節第２日'
 '第１６節第２日' '第１１節第２日' '第３４節第２日' '第３６節第２日' '第３６節第３日' '第１０節第２日' '第３６節第４日'
 '第３３節第２日' '第３１節第２日' '第１７節第３日']
gameday ['03/10(土)' '03/24(土)' '04/07(土)' '04/21(土)' '05/06(日)' '06/16(土)'
 '06/30(土)' '07/28(土)' '08/04(土)' '08/18(土)' '09/01(土)' '09/22(土・

In [ ]:
#前処理

"""list_drop = ['stadium', 'name']
df_train_simple = df_train_all.drop(list_drop, axis=1)
df_test_simple = df_test_all.drop(list_drop, axis=1)"""

list_select = ['id', 'year', 'stage', 'match', 'gameday', 'time', 'home', 'away', 'tv', 'home_score', 'away_score', 'weather', 'temperature', 'humidity', 'capa']
df_train_simple = df_train_all[['y'] + list_select]
df_test_simple = df_test_all[list_select]

df_all_simple = {'train': df_train_simple, 'test': df_test_simple}
weekdays = {'月': 0, '火': 1, '水': 2, '木': 3, '金': 4, '土': 5, '日': 6}

for name, df in df_all_simple.items():
    df['stage'] = df['stage'].replace({'Ｊ１': 1, 'Ｊ２': 2})
    df['humidity'] = df['humidity'].str.replace('%', '').astype(int)
    df['sec'] = df['match'].str.extract('第(\d+)節').astype(int)
    df['day'] = df['match'].str.extract('第(\d+)日').astype(int)
    df['week'] = df['gameday'].str.extract('(\w)').replace(weekdays).astype(int)
    df['date'] = pd.to_datetime('2020/' + df['gameday'].str.extract('(\d+/\d+)')[0], format='%Y/%m/%d')
    df['date366'] = df['date'].dt.dayofyear
    df['holiday'] = df['gameday'].str.contains('祝|土|日').astype(int)
    df['time'] = df['time'].apply(lambda x: int(x.split(':')[0]) + int(x.split(':')[1]) / 60)
    df['rain'] = df['weather'].apply(lambda x: 1 if '雨' in x else 0)
    df['cloudy'] = df['weather'].apply(lambda x: 1 if '曇' in x else 0)
    df['sunny'] = df['weather'].apply(lambda x: 1 if '晴' in x else 0)


display(df_train_simple.head(2))
display(df_test_simple.head(2))

<ipython-input-83-4b9320d677cf>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['stage'] = df['stage'].replace({'Ｊ１': 1, 'Ｊ２': 2})
<ipython-input-83-4b9320d677cf>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['humidity'] = df['humidity'].str.replace('%', '').astype(int)
<ipython-input-83-4b9320d677cf>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:

,y,id,year,stage,match,gameday,time,home,away,tv,...,capa,sec,day,week,date,date366,holiday,rain,cloudy,sunny
0,18250,13994,2012,1,第１節第１日,03/10(土),14.066667,ベガルタ仙台,鹿島アントラーズ,スカパー／ｅ２／スカパー光／ＮＨＫ総合,...,19694,1,1,0,2020-03-10,70,1,1,0,0
1,11831,14012,2012,1,第３節第１日,03/24(土),13.050000,ベガルタ仙台,大宮アルディージャ,スカパー／ｅ２／スカパー光（Ｊ ＳＰＯＲＴＳ １）,...,19694,3,1,0,2020-03-24,84,1,1,1,0


,id,year,stage,match,gameday,time,home,away,tv,home_score,...,capa,sec,day,week,date,date366,holiday,rain,cloudy,sunny
0,15822,2014,1,第１８節第１日,08/02(土),19.066667,ベガルタ仙台,大宮アルディージャ,スカパー！／スカパー！プレミアムサービス,2,...,19694,18,1,0,2020-08-02,215,1,0,0,1
1,15840,2014,1,第２０節第１日,08/16(土),19.066667,ベガルタ仙台,清水エスパルス,スカパー！／スカパー！プレミアムサービス,3,...,19694,20,1,0,2020-08-16,229,1,0,1,0


In [ ]:
#回帰モデルの作成

list_x = ['capa', 'temperature', 'stage', 'humidity', 'sec',
          'day', 'holiday', 'week', 'date366', 'year', 'time',
          'rain', 'cloudy', 'sunny']
#残り['id', 'home', 'away', 'tv', 'home_score', 'away_score']

x = sm.add_constant(df_train_simple[list_x])
y = df_train_simple['y']

sm_j1 = sm.OLS(y,x).fit()
sm_j1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.658
Model:                            OLS   Adj. R-squared:                  0.656
Method:                 Least Squares   F-statistic:                     266.9
Date:                Thu, 08 Feb 2024   Prob (F-statistic):               0.00
Time:                        14:52:27   Log-Likelihood:                -19298.
No. Observations:                1953   AIC:                         3.863e+04
Df Residuals:                    1938   BIC:                         3.871e+04
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const       -4.013e+05   3.02e+05     -1.328      0.184   -9.94e+05    1.91e+05
capa            0.2817      0.009     32.907      0.000       0.265       0.299
temperature   -35.7535     26.162     -1.367      0.172     -87.063      15.555
stage       -7170.1764    372.783    -19.234      0.000   -7901.275   -6439.078
humidity      -24.2486      8.128     -2.983      0.003     -40.190      -8.308
sec           -48.9114     58.022     -0.843      0.399    -162.704      64.881
day          -357.6541    375.120     -0.953      0.340   -1093.335     378.027
holiday      2027.4990    358.678      5.653      0.000    1324.064    2730.934
week         -570.2528    647.936     -0.880      0.379   -1840.978     700.472
date366        20.4709      8.772      2.334      0.020       3.267      37.675
year          205.2343    150.084      1.367      0.172     -89.108     499.577
time           41.3542     60.647      0.682      0.495     -77.586     160.294
rain           90.6969    369.878      0.245      0.806    -634.703     816.097
cloudy        466.2492    310.293      1.503      0.133    -142.295    1074.793
sunny         637.2461    354.435      1.798      0.072     -57.869    1332.361
==============================================================================
Omnibus:                      436.251   Durbin-Watson:                   1.109
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2257.345
Skew:                           0.955   Prob(JB):                         0.00
Kurtosis:                       7.908   Cond. No.                     8.24e+07
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.24e+07. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:
df_train_simple['pred_y'] = sm_j1.predict(x)
pred_y = df_train_simple['pred_y']

def calc_rmse(x,y):
  return math.sqrt(np.mean((x-y)**2))
rmse_train = calc_rmse(pred_y, y)

rmse_train

<ipython-input-89-490772d08479>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_simple['pred_y'] = sm_j1.predict(x)


4733.747104009067